In [30]:
import streamlit as st
import pandas as pd
import plotly.express as px

#### What we need to include in the app: ####
- at least one `st.header` with text
- at least one `plotly-express` histogram using `[st.write][https://docs.streamlit.io/library/api-reference/write-magic/st.write][st.plotly_chart](https://docs.streamlit.io/library/api-reference/charts/st.plotly_chart)`
- at least one `plotly-express` scatterplot using `[st.write][https://docs.streamlit.io/library/api-reference/write-magic/st.write][st.plotly_chart](https://docs.streamlit.io/library/api-reference/charts/st.plotly_chart)`
- at least one checkbox that uses `[st.checkbox](https://docs.streamlit.io.library/api-reference/widgets/st.checkbox)` that changes the behavior of any of the above components

In [31]:
df = pd.read_csv('moved_vehicles_us.csv')

### Overview of data

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [33]:
df.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


### Preprocessing data

In [34]:
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%Y-%m-%d') # Converting dates to datetime data type
df.isna().sum() # Getting a count of missing values by column

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

The `is_4wd` column appears to contain the value of `1.0` if the vehicle was listed to have four wheel drive, and a missing value if not. This will be confirmed to get the unique values, including missing from this column, and if there are only the two aforementioned values, we will replace missing values with `0` (zero) and convert them to integers to act as a Bulian true or false, 1 = true, 0 = false. This doesn't mean that all the vehicles listed with a 0 (zero) do not have four wheel drive, but that they were not advertised to have it.

In [35]:
df['is_4wd'].value_counts(dropna=False)

NaN    25953
1.0    25572
Name: is_4wd, dtype: int64

In [36]:
df['is_4wd'] = df['is_4wd'].fillna(0)
df = df.astype({'is_4wd':'int64'})
df['is_4wd'].value_counts(dropna=False)

0    25953
1    25572
Name: is_4wd, dtype: int64

In [37]:
makemodel = df["model"].str.split(" ", n = 1, expand = True)
df['make'] = makemodel[0]
df['model'] = makemodel[1]
df.rename(columns={'model_year':'year', 'paint_color':'color'}, inplace=True)
df = df[['price', 'year', 'make', 'model', 'condition', 'cylinders', 'fuel','odometer', 'transmission', 'type', 'color', 'is_4wd', 'date_posted', 'days_listed']]

### Streamlit coding

In [38]:
st.header('Exploratory Analysis of Auto Listings')
st.write('''
Default display of vehicles only includes those listed for 30 days or less. In order to see all listings, including those listed for more than 30 days, click the checkbox below.
''')
old_listings = st.checkbox('Show cars listed over 30 days')

In [39]:
# Checkbox to include vehicles posted for over 30 days
if not old_listings:
    df = df[df.days_listed<=30]

##### histogram for price(y) based on year(x) (slider for year)

In [40]:
# Selectbox for histogram
hist_list = ['condition', 'fuel', 'transmission', 'type', 'is_4wd']
hist_select = st.selectbox('Filter for average price', hist_list)

In [41]:
# Slider for year, limits then slider
min_year, max_year = int(df['year'].min()), int(df['year'].max())

year_range = st.slider(
    'Choose years',
    value=(min_year, max_year), min_value=min_year, max_value=max_year )

In [42]:
year_act_range = list(range(year_range[0],year_range[1]+1))
year_df = df[(df.type == type_choice) & (df.year.isin(list(year_act_range)))]

In [43]:
fig1 = px.histogram(year_df, x='year', y='price', histfunc='avg', color=hist_select)

fig1.update_layout(title='<b>Average Price by Year</b>')

st.plotly_chart(fig1)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [44]:
fig1.show()

##### scatterplot for odometer(y) based on price(x) (slider for price)

In [45]:
# Selectbox for scatterplot
scat_list = ['condition', 'fuel', 'transmission', 'type', 'is_4wd']
scat_select = st.selectbox('Filter for mileage by price', scat_list)

In [46]:
# Slider for price, limits then slider
min_price, max_price = int(df['price'].min()), int(df['price'].max())

price_range = st.slider(
    'Set price range',
    value=(min_price, max_price), min_value=min_price, max_value=max_price )

In [47]:
price_act_range = list(range(price_range[0],price_range[1]+1))
price_df = df[(df.type == type_choice) & (df.price.isin(list(price_act_range)))]

In [48]:
fig2 = px.scatter(price_df, x='price', y='odometer', color=scat_select)

fig2.update_layout(title='<b>Mileage by Price (USD)</b>')

st.plotly_chart(fig2)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [49]:
fig2.show()

##### boxplot for odometer (y) based on cylinders (slider of cylinders)

In [50]:
# Selectbox for box chart
box_list = ['condition', 'fuel', 'transmission', 'type', 'is_4wd']
box_select = st.selectbox('Filter for mileage by cylinder count', box_list)

In [51]:
# Slider of cylinders, limits then slider
min_cyl, max_cyl = int(df['cylinders'].min()), int(df['cylinders'].max())

cyl_range = st.slider(
    'Specify number of cylinders',
    value=(min_cyl, max_cyl), min_value=min_cyl, max_value=max_cyl )

In [52]:
cyl_act_range = list(range(cyl_range[0],cyl_range[1]+1))
cyl_df = df[(df.type == type_choice) & (df.cylinders.isin(cyl_act_range))]

In [53]:
fig3 = px.box(cyl_df, x='cylinders', y='odometer', color=box_select)

fig3.update_layout(title='<b>Distribution of Mileage by Cylinder Count</b>')

st.plotly_chart(fig3)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [54]:
fig3.show()